<a href="https://colab.research.google.com/github/otumina-collab/POO-Actividad_N-_6/blob/main/Actividad_N%C2%B06_8_5_POO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#Actividad N°6 - 8.5.
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime, date
import json
import os
# 1. MODELO DE PROGRAMACIÓN ORIENTADA A OBJETOS (POO)


class Huesped:
    """Define un huésped del hotel."""
    def __init__(self, nombres, apellidos, documento_identidad, fecha_ingreso):
        self.nombres = nombres
        self.apellidos = apellidos
        self.documento_identidad = documento_identidad
        self.fecha_ingreso = fecha_ingreso

    def to_dict(self):
        """Convierte el objeto Huesped a un diccionario para persistencia."""
        return {
            "nombres": self.nombres,
            "apellidos": self.apellidos,
            "documento_identidad": self.documento_identidad,
            "fecha_ingreso": self.fecha_ingreso.isoformat()
        }

    @classmethod
    def from_dict(cls, data):
        """Crea un objeto Huesped desde un diccionario."""
#Asegurar que documento_identidad sea int, ya que en el UI puede venir como string
        documento = int(data["documento_identidad"]) if isinstance(data["documento_identidad"], str) else data["documento_identidad"]
        return cls(
            data["nombres"],
            data["apellidos"],
            documento,
            datetime.fromisoformat(data["fecha_ingreso"])
        )

class Habitacion:
    """Define una habitación del hotel."""
    def __init__(self, numero, disponible, precio_dia):
        self.numero = numero
        self.disponible = disponible
        self.precio_dia = precio_dia
        self.huesped = None

    def __str__(self):
        estado = "Disponible" if self.disponible else f"Ocupada por {self.huesped.nombres} {self.huesped.apellidos} (Ingreso: {self.huesped.fecha_ingreso.strftime('%Y-%m-%d')})"
        return f"Habitación {self.numero} (${self.precio_dia:,.0f} / día) - {estado}"

    def to_dict(self):
        """Convierte el objeto Habitacion a un diccionario para persistencia."""
        data = {
            "numero": self.numero,
            "disponible": self.disponible,
            "precio_dia": self.precio_dia
        }
        if self.huesped:
            data["huesped"] = self.huesped.to_dict()
        return data

    @classmethod
    def from_dict(cls, data):
        """Crea un objeto Habitacion desde un diccionario."""
        habitacion = cls(
            data["numero"],
            data["disponible"],
            data["precio_dia"]
        )
        if "huesped" in data and data["huesped"] is not None:
            habitacion.huesped = Huesped.from_dict(data["huesped"])
        return habitacion

class Hotel:
    """Gestiona el conjunto de habitaciones y las operaciones del hotel."""
    def __init__(self):
        self.lista_habitaciones = []
        if not self.load_data():

            for i in range(1, 11):
                precio = 120000.0 if i <= 5 else 160000.0
                self.lista_habitaciones.append(Habitacion(i, True, precio))

    def save_data(self):
        """Guarda el estado actual de las habitaciones y huéspedes en un archivo JSON."""
        data_to_save = [hab.to_dict() for hab in self.lista_habitaciones]
        try:
            with open('hotel_data.json', 'w', encoding='utf-8') as f:
                json.dump(data_to_save, f, ensure_ascii=False, indent=4)

        except IOError as e:
            print(f"Error al guardar los datos: {e}")

    def load_data(self):
        """Carga el estado del hotel desde un archivo JSON."""
        loaded_rooms = []
        if not os.path.exists('hotel_data.json') or os.stat('hotel_data.json').st_size == 0:

            return False
        try:
            with open('hotel_data.json', 'r', encoding='utf-8') as f:
                data_from_file = json.load(f)
            for room_data in data_from_file:
                loaded_rooms.append(Habitacion.from_dict(room_data))
            self.lista_habitaciones = loaded_rooms

            return True
        except FileNotFoundError:
            print("Error: Archivo 'hotel_data.json' no encontrado. Se inicializará con datos predeterminados.")
            return False
        except json.JSONDecodeError:
            print("Error: No se pudo leer el archivo 'hotel_data.json'. Archivo corrupto o formato incorrecto. Se inicializará con datos predeterminados.")
            return False
        except Exception as e:
            print(f"Error inesperado al cargar los datos: {e}. Se inicializará con datos predeterminados.")
            return False


    def buscar_habitacion(self, numero):
        """Busca una habitación por su número."""
        for hab in self.lista_habitaciones:
            if hab.numero == numero:
                return hab
        return None

    def registrar_ingreso(self, numero, nombres, apellidos, documento, fecha_ingreso):
        """Registra el ingreso de un huésped."""
        hab = self.buscar_habitacion(numero)

        if hab is None:
            return "Error: El número de habitación no existe."
        if not hab.disponible:
            return f"Error: La habitación {numero} ya está ocupada."

        # Validar campos obligatorios
        if not all([nombres, apellidos, documento, fecha_ingreso]):
             return "Error: Todos los campos de huésped y fecha de ingreso son obligatorios."

        try:
            documento = int(documento)
        except ValueError:
            return "Error: El documento de identidad debe ser un número."

        nuevo_huesped = Huesped(nombres, apellidos, documento, fecha_ingreso)
        hab.huesped = nuevo_huesped
        hab.disponible = False

        return f"Registro exitoso: Huésped '{nombres} {apellidos}' ingresado en Habitación {numero}."

    def registrar_salida(self, numero, fecha_salida):
        """Calcula el pago y registra la salida del huésped."""
        hab = self.buscar_habitacion(numero)

        if hab is None or hab.disponible:
            return "Error: La habitación no está ocupada o el número es incorrecto."

        fecha_ingreso = hab.huesped.fecha_ingreso

        # Validar que la fecha de salida sea mayor que la de ingreso
        if fecha_salida <= fecha_ingreso.date():
             return "Error: La fecha de salida debe ser posterior a la de ingreso."

        # Cálculo de días de alojamiento
        dias_alojamiento = (fecha_salida - fecha_ingreso.date()).days
        total_a_pagar = dias_alojamiento * hab.precio_dia

        # Liberar la habitación
        huesped_saliente = hab.huesped
        hab.huesped = None
        hab.disponible = True

        return (f"Salida registrada para la Habitación {numero}.\n"
                f"Huésped: {huesped_saliente.nombres} {huesped_saliente.apellidos}\n"
                f"Días de Alojamiento: {dias_alojamiento}\n"
                f"Total a Pagar: ${total_a_pagar:,.0f}")

#2. INTERFAZ GRÁFICA PARA GOOGLE COLAB (ipywidgets)

hotel = Hotel()

#Componentes Comunes
output_area = widgets.Output()
#Consulta de Habitaciones
habitaciones_label = widgets.HTML(value="<h2>Estado Actual de las Habitaciones</h2>")
habitaciones_output = widgets.VBox([], layout=widgets.Layout(align_items='center'))

def actualizar_lista_habitaciones():
    """Genera la lista interactiva de habitaciones en un formato de cuadrícula."""
    room_widgets = []
    for hab in hotel.lista_habitaciones:
        if hab.disponible:
            background_color_room = '#7CFC00'  # LawnGreen (a nice green)
            text_color_room = 'white'
            estado_texto = "Disponible"
        else:
            background_color_room = '#FF6347'  # Tomato (a reddish color)
            text_color_room = 'white'
            estado_texto = f"Ocupada (Huésped: {hab.huesped.nombres} / Ingreso: {hab.huesped.fecha_ingreso.strftime('%Y-%m-%d')})"

        room_html = widgets.HTML(f'<div style="text-align: center; color: {text_color_room};">Hab. {hab.numero}: {estado_texto}</div>')

        # Add background_color to the layout of the VBox
        room_container = widgets.VBox([room_html], layout=widgets.Layout(
            border='1px solid lightgray',
            padding='2px',
            margin='1px',
            width='180px',
            align_items='center',
            background_color=background_color_room
        ))
        room_widgets.append(room_container)

    grid_rows = []
    for i in range(0, len(room_widgets), 5):
        row_widgets = room_widgets[i:i+5]
        if row_widgets:
            grid_rows.append(widgets.HBox(row_widgets, layout=widgets.Layout(justify_content='center')))

    habitaciones_output.children = grid_rows

#Actualizar la lista de habitaciones disponibles para el Dropdown
    habs_disponibles = [str(h.numero) for h in hotel.lista_habitaciones if h.disponible]
    hab_ingreso_select.options = habs_disponibles
#Registro de Huésped
reg_huesped_label = widgets.HTML(value="<h3>Registrar Ingreso de Huésped</h3>")
hab_ingreso_select = widgets.Dropdown(description='Habitación:', options=[], value=None)
nombre_ingreso = widgets.Text(description='Nombre(s):')
apellido_ingreso = widgets.Text(description='Apellido(s):')
doc_ingreso = widgets.Text(description='Documento ID:', placeholder='Solo números')
fecha_ingreso = widgets.DatePicker(description='Fecha Ingreso:', value=date.today(), disabled=False)
btn_registrar_ingreso = widgets.Button(description='Registrar Ingreso', button_style='success') # Set to 'success'
ingreso_output = widgets.Label(value="")

def on_ingreso_click(b):
    """Maneja el evento de registrar ingreso."""
    with output_area:
        clear_output()
        try:
            hab_numero = int(hab_ingreso_select.value)
        except (ValueError, TypeError):
             print("Seleccione una habitación disponible.")
             return

        resultado = hotel.registrar_ingreso(
            hab_numero,
            nombre_ingreso.value,
            apellido_ingreso.value,
            doc_ingreso.value,
            datetime.combine(fecha_ingreso.value, datetime.min.time())
        )
        ingreso_output.value = resultado
        hotel.save_data()
        actualizar_lista_habitaciones()
#Limpiar campos después de un registro exitoso
        if "Registro exitoso" in resultado:
            nombre_ingreso.value = ""
            apellido_ingreso.value = ""
            doc_ingreso.value = ""
            fecha_ingreso.value = date.today()

btn_registrar_ingreso.on_click(on_ingreso_click)

tab_ingreso_content = widgets.VBox([
    habitaciones_label,
    widgets.VBox([habitaciones_output], layout=widgets.Layout(border='2px solid lightgray', padding='5px', align_items='center')),
    reg_huesped_label,
    hab_ingreso_select,
    nombre_ingreso,
    apellido_ingreso,
    doc_ingreso,
    fecha_ingreso,
    btn_registrar_ingreso,
    ingreso_output
])

#Registrar Salida de Huéspedes
#Búsqueda y Datos
salida_label = widgets.HTML(value="<h2>Registrar Salida de Huésped</h2>")
hab_salida_select = widgets.Dropdown(description='Habitación:', options=[str(i) for i in range(1, 11)], value='1')
btn_consultar_salida = widgets.Button(description='Consultar Habitación', button_style='primary')
huesped_info = widgets.Label(value="Información del Huésped: N/A")
fecha_salida = widgets.DatePicker(description='Fecha Salida:', value=date.today(), disabled=False)
fecha_salida_output = widgets.Label(value="")
total_output = widgets.Label(value="")

#Botones de Acción
btn_calcular_total = widgets.Button(description='Calcular Total a Pagar', button_style='warning', disabled=True)
btn_registrar_salida = widgets.Button(description='Registrar Salida', button_style='success', disabled=True) # Set to 'success'
salida_output = widgets.Label(value="")

def on_consultar_salida_click(b):
    """Maneja el evento de consultar habitación para salida."""
    with output_area:
        clear_output()
        hab_numero = int(hab_salida_select.value)
        hab = hotel.buscar_habitacion(hab_numero)

        total_output.value = ""
        btn_calcular_total.disabled = True
        btn_registrar_salida.disabled = True

        if hab is None or hab.disponible:
            huesped_info.value = "Habitación no ocupada."
            fecha_salida_output.value = ""
            return

#Habitación ocupada
        huesped_info.value = (
f"Habitación {hab_numero} (Precio: ${hab.precio_dia:,.0f}): "
f"Ocupada por {hab.huesped.nombres} {hab.huesped.apellidos} "
f"(Ingreso: {hab.huesped.fecha_ingreso.strftime('%Y-%m-%d')})")

#Habilitar el cálculo después de la consulta
        btn_calcular_total.disabled = False

btn_consultar_salida.on_click(on_consultar_salida_click)

def on_calcular_total_click(b):
    """Maneja el evento de calcular el total a pagar."""
    with output_area:
        clear_output()
        hab_numero = int(hab_salida_select.value)
        hab = hotel.buscar_habitacion(hab_numero)

        fecha_sal = fecha_salida.value
        fecha_ingreso_date = hab.huesped.fecha_ingreso.date()

        if fecha_sal <= fecha_ingreso_date:
            fecha_salida_output.value = "Error: La fecha de salida debe ser posterior a la de ingreso."
            total_output.value = ""
            btn_registrar_salida.disabled = True
            return

        fecha_salida_output.value = ""
        dias_alojamiento = (fecha_sal - fecha_ingreso_date).days
        total_a_pagar = dias_alojamiento * hab.precio_dia

        total_output.value = (
f"Cálculo: {dias_alojamiento} días de alojamiento. "
f"Total a pagar: ${total_a_pagar:,.0f}")

#Habilitar el botón de registro después del cálculo exitoso
        btn_registrar_salida.disabled = False

btn_calcular_total.on_click(on_calcular_total_click)

def on_registrar_salida_click(b):
    """Maneja el evento de registrar salida."""
    with output_area:
        clear_output()
        hab_numero = int(hab_salida_select.value)

        resultado = hotel.registrar_salida(hab_numero, fecha_salida.value)

        salida_output.value = resultado
        huesped_info.value = "Información del Huésped: N/A"
        total_output.value = ""
        btn_calcular_total.disabled = True
        btn_registrar_salida.disabled = True
        hotel.save_data()
        actualizar_lista_habitaciones()

btn_registrar_salida.on_click(on_registrar_salida_click)

tab_salida_content = widgets.VBox([
    salida_label,
    hab_salida_select,
    btn_consultar_salida,
    huesped_info,
    fecha_salida_output,
    fecha_salida,
    widgets.HBox([btn_calcular_total, btn_registrar_salida]),
    total_output,
    salida_output
])

#Contenedor Principal

main_tabs = widgets.Tab()
main_tabs.children = [tab_ingreso_content, tab_salida_content]
main_tabs.set_title(0, 'Consultar Habitaciones')
main_tabs.set_title(1, 'Salida de Huéspedes')

# Mostrar la interfaz al usuario
def run_app():
    actualizar_lista_habitaciones()
    hotel_title = widgets.HTML("<h1>Hotel</h1>") # New line for the title
    display(hotel_title, main_tabs, output_area) # Display the title along with others

# Para ejecutar en Colab, solo llame a run_app()
run_app()

HTML(value='<h1>Hotel</h1>')

Output()